### Carga de dados, armazenagem e formatos de arquivo

#### Lendo e escrevendo dados em formato-texto

O pandas tem uma série de funções para ler dados tabulares na
forma de um objeto DataFrame. A Tabela abaixo sintetiza algumas
delas, embora *read_csv* e *read_table* provavelmente sejam aquelas
que você usará com mais frequência.

#### Tabela – Funções de parsing do pandas
Função | Descrição
-------|----------
read_csv | Carrega dados delimitados de um arquivo, um URL ou um objeto do tipo arquivo; utiliza vírgula como delimitador default
read_table | Carrega dados delimitados de um arquivo, um URL ou um objeto do tipo arquivo; utiliza tabulação ('\t') como delimitador default
read_fwf | Lê dados em formato de coluna com tamanho fixo (isto é, sem delimitadores)
read_clipboard | Versão de read_table que lê dados da área de transferência (clipboard); é útil para converter tabelas de páginas web
read_excel | Lê dados tabulares de um arquivo Excel XLS ou XLSX
read_hdf | Lê arquivos HDF5 escritos pelo pandas
read_html | Lê todas as tabelas que se encontram no documento HTML especificado
read_json | Lê dados de uma representação em string JSON (JavaScript Object Notation, ou Notação de Objetos JavaScript)
read_msgpack | Lê dados codificados pelo pandas no formato binário MessagePack
read_pickle | Lê um objeto arbitrário armazenado no formato pickle de Python
read_sas | Lê um conjunto de dados SAS armazenado em um dos formatos personalizados do sistema SAS
read_sql | Lê o resultado de uma consulta SQL (usando SQLAlchemy) na forma de um DataFrame do pandas
read_stata | Lê um conjunto de dados no formato de arquivo Stata
read_feather | Lê o formato de arquivo binário Feather

Os argumentos opcionais dessas funções podem se
enquadrar em algumas categorias:

* *Indexação*
 * Para tratar uma ou mais colunas como o DataFrame devolvido, e
se os nomes das colunas devem ser obtidos do arquivo, do
usuário ou de nenhum deles.

* *Inferência de tipos e conversão de dados*
 * Inclui as conversões de valores definidas pelo usuário e uma lista
personalizada de marcadores de valores ausentes.

* *Parsing de data e hora*
 * Inclui recursos de combinação, entre eles, combinação de
informações de data e hora espalhadas em várias colunas em uma
única coluna no resultado.

* *Iteração*
 * Suporte para iteração em partes de arquivos bem grandes.

* *Problemas com dados sujos*
 * Pular linhas ou um rodapé, comentários ou outras pequenas
informações como dados numéricos com vírgulas para separar
milhares.

In [25]:
import pandas as pd
import numpy as np

Considerando que os dados estão delimitados por vírgula, podemos
usar *read_csv* para lê-los em um DataFrame:

In [2]:
path = '/content/drive/MyDrive/Análise de Dados/CSVs e TXTs/ex1.csv'
df = pd.read_csv(path)
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Também poderíamos ter usado *read_table* e especificado o
delimitador:

In [3]:
pd.read_table(path, sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Um arquivo nem sempre terá uma linha de cabeçalho.Para ler esse arquivo, temos duas opções. Podemos permitir que o
pandas atribua nomes default para as colunas ou podemos, nós
mesmos, especificar os nomes:

In [4]:
path = '/content/drive/MyDrive/Análise de Dados/CSVs e TXTs/ex2.csv'
pd.read_csv(path, header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
pd.read_csv(path, names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suponha que quiséssemos que a coluna *message* fosse o índice do
DataFrame devolvido. Podemos informar que queremos a coluna no
índice 4 ou de nome *'message'* utilizando o argumento *index_col*:

In [6]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv(path, names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


Caso você queira compor um índice hierárquico a partir de várias
colunas, passe uma lista de números ou de nomes de colunas:

In [7]:
path = '/content/drive/MyDrive/Análise de Dados/CSVs e TXTs/ex3.csv'
parsed = pd.read_csv(path, index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

Em alguns casos, uma tabela pode não ter um delimitador fixo,
usando espaço em branco ou outro padrão para separar os campos.

Embora você possa fazer algumas manipulações manualmente, os
campos, nesse caso, estão separados por uma quantidade variável
de espaços em branco. Em situações como essa, é possível passar
uma expressão regular como um delimitador para *read_table*. Nesse
exemplo, podemos usar a expressão regular *\s+* , de modo que
teríamos:

In [8]:
path = '/content/drive/MyDrive/Análise de Dados/CSVs e TXTs/ex4.txt'
result = pd.read_table(path, sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


Como havia um nome de coluna a menos que o número de linhas
de dados, *read_table* infere que a primeira coluna deve ser o índice
do DataFrame nesse caso especial.

As funções de parser têm muitos argumentos adicionais para ajudar
você a lidar com a grande variedade de possíveis formatos de
arquivo excepcionais. Por
exemplo, podemos ignorar a primeira, a terceira e a quarta linhas de
um arquivo usando *skiprows*:

In [9]:
path = '/content/drive/MyDrive/Análise de Dados/CSVs e TXTs/ex5.csv'
pd.read_csv(path, skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Lidar com valores ausentes é uma parte importante e, em geral,
com certas nuances, no processo de parsing do arquivo. Dados
ausentes geralmente não estão presentes (são strings vazias) ou
estão marcados com algum valor de *sentinela*. Por padrão, o pandas
utiliza um conjunto de sentinelas que ocorrem usualmente, como **NA**
e **NULL** :

In [10]:
path = '/content/drive/MyDrive/Análise de Dados/CSVs e TXTs/ex6.csv'
result = pd.read_csv(path)
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [11]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


A opção *na_values* pode aceitar uma lista ou um conjunto de strings a
serem considerados como valores ausentes:

In [12]:
result = pd.read_csv(path, na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Sentinelas **NA** diferentes podem ser especificadas para cada coluna
em um dicionário:

In [13]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv(path, na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Tabela – Alguns argumentos das funções read_csv/read_table
Argumento | Descrição
----------|-------------
**Path** | String que indica o local no sistema de arquivos, o URL ou um
     | objeto do tipo arquivo
**sep ou delimiter** | Sequência de caracteres ou uma expressão regular a  
 | ser usada para separar campos em cada linha
**header** | Número da linha a ser usada como nomes de coluna; o default é
 | 0 (primeira linha), mas deverá ser None se não houver linha de
 | cabeçalho
**index_col** | Números ou nomes de coluna a serem usados como índice das
|linhas no resultado; pode ser um único nome/número ou uma
|lista deles se for um índice hierárquico
**names** | Lista de nomes de colunas para o resultado; pode ser combinado
|com header=None
**skiprows** | Número de linhas a serem ignoradas no início do arquivo ou   
|lista de números de linhas (começando de 0) a serem ignoradas.
**na_values** | Sequência de valores a serem substituídos por NA.
**comment** | Caractere(s) para separar comentários no final das linhas
**parse_dates** | Tenta fazer parse de dados para datetime; o default é
| False. Se for True, tentará fazer parse de todas as colunas. Caso 
|contrário, poderá especificar uma lista de números ou nomes de colunas
|para o parse. Se o elemento da lista for uma tupla ou uma lista,
|combinará várias colunas e fará o parse para data (por exemplo,
|se a data/hora estiverem separadas em duas colunas).
**keep_date_col** | Se as colunas forem reunidas para parse de data, mantém as
|colunas unidas; o default é False.
**converters** | Dicionário contendo o número ou o nome de colunas mapeados
|para funções (por exemplo, {'foo': f} aplica a função f em todos os
|valores da coluna 'foo').
**dayfirst** | Ao fazer parsing de datas potencialmente ambíguas, trata-as
|como estando no formato internacional (por exemplo, 7/6/2012 ->
|7 de junho de 2012); o default é False.
**date_parser** | Função a ser usada para parse de datas.
**nrows** | Número de linhas a serem lidas no início do arquivo.
**iterator** | Devolve um objeto TextParser para ler o arquivo aos poucos.
**chunksize** | Para iteração, é o tamanho das partes do arquivo.
**skip_footer** | Número de linhas a serem ignoradas no final do arquivo.
**verbose** | Exibe várias informações de saída do parser, como o número de
|valores ausentes em colunas não numéricas.
**encoding** | Codificação de texto para Unicode (por exemplo, 'utf-8' para texto
|codificado em UTF-8).
**squeeze** | Se os dados sujeitos a parsing contiverem apenas uma coluna,
|devolve uma Series.
**thousands** | Separador de milhar (por exemplo, ',' ou '.').


### Lendo arquivos-texto em partes
Ao processar arquivos bem grandes ou descobrir o conjunto certo
de argumentos para processar corretamente um arquivo grande,
talvez você queira ler somente uma pequena parte ou iterar por
porções menores do arquivo.

Antes de observar um arquivo grande, vamos alterar as
configurações do pandas a fim de que a exibição dos dados seja
mais compacta:

In [ ]:
path = '/content/drive/MyDrive/Análise de Dados/CSVs e TXTs/ex7.csv'
pd.options.display.max_rows = 10
result = pd.read_csv(path)
result

,one,two,three,four,key
0,0.125109,-2.278962,-1.011600,-2.665703,L
1,0.679354,0.927139,-2.130432,-4.289954,O
2,-1.487000,1.944473,-0.015792,-2.164808,E
3,1.080752,0.047718,1.564583,-0.275413,M
4,-4.713821,-0.412510,2.671375,-1.373800,E
...,...,...,...,...,...
9995,-2.473530,1.693003,-0.582499,2.297526,M
9996,-0.962632,1.170375,-5.752872,0.263912,M
9997,-3.895008,-1.961017,1.020955,3.110367,Q
9998,1.692233,0.096650,1.637344,-5.041235,O


Se você quiser ler apenas uma quantidade pequena de linhas
(evitando ler o arquivo todo), especifique isso usando *nrows*:

In [ ]:
pd.read_csv(path, nrows=5)

,one,two,three,four,key
0,0.125109,-2.278962,-1.011600,-2.665703,L
1,0.679354,0.927139,-2.130432,-4.289954,O
2,-1.487000,1.944473,-0.015792,-2.164808,E
3,1.080752,0.047718,1.564583,-0.275413,M
4,-4.713821,-0.412510,2.671375,-1.373800,E


Para ler um arquivo em partes, especifique uma quantidade de
linhas para *chunksize*:

In [ ]:
chunker = pd.read_csv(path, chunksize=1000)
chunker

O objeto *TextParser* devolvido por *read_csv* permite iterar pelas partes
do arquivo de acordo com o *chunksize*. Por exemplo, podemos iterar
por *ex7.csv*, agregando os contadores de valores na coluna *'key'*,
assim:

In [ ]:
tot = pd.Series([])

for piece in chunker:
  tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [ ]:
tot[: 10]

M    1705.0
E    1694.0
O    1682.0
Q    1680.0
L    1620.0
X    1619.0
dtype: float64

*TextParser* também oferece um método get_chunk que permite ler
partes de tamanho arbitrário.

### Escrevendo dados em formato-texto

Os dados também podem ser exportados para um formato com
delimitador. Vamos considerar um dos arquivos CSV que lemos
antes:

In [16]:
path = '/content/drive/MyDrive/Análise de Dados/CSVs e TXTs/ex6.csv'
data = pd.read_csv(path)
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Usando o método *to_csv* de DataFrame, podemos escrever os dados
separados por vírgula em um arquivo:

In [17]:
data.to_csv('ex1_saida.csv')

In [18]:
import sys

data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Valores ausentes aparecem como strings vazias na saída. Você
pode representá-las com outro valor de sentinela:

In [19]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


Sem outras opções especificadas, os rótulos tanto das linhas quanto
das colunas são escritos. Ambos podem ser desativados:

In [21]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


Também podemos escrever somente um subconjunto das colunas,
em uma ordem de sua preferência:

In [22]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


Uma Series também tem um método *to_csv*:

In [23]:
dates = pd.date_range('1/1/2000', periods=7)
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')

In [26]:
ts = pd.Series(np.arange(7), index=dates)
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int64

In [27]:
ts.to_csv('ex2_saida.csv')

### Trabalhando com formatos delimitados
É possível carregar a maioria dos formatos de dados tabulares de
disco usando funções como *pandas.read_table* . Em alguns casos,
porém, um pouco de processamento manual talvez seja necessário.
Não é incomum receber um arquivo com uma ou mais linhas
malformadas que poderão confundir *read_table*. Para demonstrar ouso das ferramentas básicas, considere um pequeno arquivo CSV:

In [34]:
path = '/content/drive/MyDrive/Análise de Dados/CSVs e TXTs/ex8.csv'

Para qualquer arquivo com um único caractere como delimitador,
podemos usar o módulo embutido csv de Python. Para usá-lo, passe
qualquer arquivo aberto ou um objeto do tipo arquivo para *csv.reader*:

In [28]:
import csv

In [35]:
f = open(path)
aux = csv.reader(f)
reader = [i for i in aux]
f.close()

Iterar pelo reader como um arquivo produz tuplas de valores com
qualquer caractere de aspas removido:

In [36]:
for line in reader:
  print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


A partir daí, cabe a você fazer a manipulação necessária para deixar
os dados no formato em que precisar. Vamos executar essa tarefa
passo a passo. Inicialmente vamos ler o arquivo em uma lista de
linhas:

In [37]:
with open(path) as f:
  lines = list(csv.reader(f))

In [41]:
header, values = lines[0], lines[1:]
print(header, values)

['a', 'b', 'c'] [['1', '2', '3'], ['1', '2', '3']]


Então podemos criar um dicionário de colunas de dados usando
uma dictionary comprehension (abrangência de dicionário) e a
expressão zip(*values), que faz a transposição das linhas para as
colunas:

In [42]:
data_dict = {key: value for key, value in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

Os arquivos CSV têm muitas variantes distintas. Para definir um
novo formato com um delimitador, uma convenção de aspas para
strings ou um finalizador de linha diferentes, definimos uma
subclasse simples de *csv.Dialect*:

In [48]:
class my_dialect(csv.Dialect):
  lineterminator = '\n'
  delimiter = ';'
  quotechar = '"'
  quoting = csv.QUOTE_MINIMAL

with open(path, mode='r') as f:
  reader = csv.reader(f, dialect=my_dialect)
  print(list(reader))

[['a,"b","c"'], ['1,"2","3"'], ['1,"2","3"']]


### Tabela – Opções de dialetos de CSV
Argumento | Descrição
----------|-----------
**delimiter** | String de um caractere para separar os campos; o default é ','.
**lineterminator** | Finalizador de linha para escrita; o default é '\r\n'. O reader ignora isso e reconhece finalizadores de linha em plataformas diferentes.
**quotechar** | Caractere de aspas para campos com caracteres especiais (como um delimitador); o default é '"'.
**quoting** | Convenção para aspas. As opções incluem csv.QUOTE_ALL 
| (aspas em todos os campos), csv.QUOTE_MINIMAL (somente    
|campos com caracteres especiais, como o delimitador), csv.| QUOTE_NONNUMERIC e csv.QUOTE_NONE (sem aspas). Consulte a | documentação de Python para ver todos os detalhes. O default é QUOTE_MINIMAL.
**skipinitialspace** | Ignora espaços em branco depois de cada delimitador; o default é False.
**doublequote** | Como lidar com o caractere de aspas em um |campo; se for True, é duplo (consulte a documentação   
|online para ver todos osdetalhes e o comportamento).
**escapechar** | String para escapar o delimitador se quoting estiver definido com csv.QUOTE_NONE; o default é desativado.

OBS: *Para arquivos com delimitadores contendo vários caracteres fixos ou mais
complexos, não será possível utilizar o módulo csv. Nesses casos, será
necessário fazer a separação de linhas e outras tarefas de limpeza usando o
método split de strings ou o método de expressão regular re.split.*



Para *escrever* arquivos com delimitadores manualmente, podemos
usar *csv.writer*. Ele aceita um objeto de arquivo aberto, com
permissão para escrita, além das mesmas opções de dialeto e de formato de *csv.reader*:

In [50]:
with open('mydata.csv', 'w') as f:
  write = csv.writer(f, dialect=my_dialect)
  write.writerow(('one', 'two', 'three'))
  write.writerow(('1', '2', '3'))
  write.writerow(('4', '5', '6'))
  write.writerow(('7', '8', '9'))

### Dados JSON
O JSON (abreviatura de JavaScript Object Notation, ou Notação de
Objetos JavaScript) tornou-se um dos formatos padrões para envio
de dados em requisições HTTP entre navegadores web e outras
aplicações. É um formato de dados muito mais livre que um formato
de texto tabular como o CSV. Eis um exemplo:

In [51]:
obj = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
            {"name": "Katie", "age": 38,
             "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

O JSON é quase um código Python válido, com a exceção de seuvalor nulo *null* e outras nuances (como não permitir vírgulas no final
das listas). Os tipos básicos são objetos (dicionários), arrays (listas),
strings, números, booleanos e nulls. Todas as chaves em um objeto
devem ser strings. Há várias bibliotecas Python para ler e escrever
dados JSON. Usarei *json* nesta seção, pois ela está incluída na
biblioteca-padrão de Python. Para converter uma string JSON em
formato Python, utilize *json.loads*:

In [52]:
import json

In [54]:
result = json.loads(obj)
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 30, 'name': 'Scott', 'pets': ['Zeus', 'Zuko']},
  {'age': 38, 'name': 'Katie', 'pets': ['Sixes', 'Stache', 'Cisco']}]}

*json.dumps*, por outro lado, converte um objeto Python de volta para
JSON:

In [56]:
asjson = json.dumps(result)

O modo como você converte um objeto JSON ou uma lista de
objetos em um DataFrame ou em outra estrutura de dados para
análise é uma tarefa que caberá a você. De forma conveniente, é
possível passar uma lista de dicionários (que eram anteriormente
objetos JSON) para o construtor de DataFrame e selecionar um
subconjunto dos campos de dados:

In [57]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


*pandas.read_json* pode converter automaticamente conjuntos de dados
JSON organizados de modo específico em uma Series ou um DataFrame. As opções default para *pandas.read_json* supõem que cada objeto no
array JSON seja uma linha da tabela:

In [59]:
path = '/content/drive/MyDrive/Análise de Dados/CSVs e TXTs/ex9.json'
data = pd.read_json(path)
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


Se precisar exportar dados do pandas para JSON, uma maneira é
usar os métodos *to_json* em Series e em DataFrame:

In [60]:
data.to_json()

'{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}'

In [61]:
data.to_json(orient='records')

'[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]'

### Interagindo com APIs web
Muitos sites têm APIs públicas que oferecem feeds de dados
usando JSON ou outro formato. Há várias maneiras de acessaressas APIs a partir de Python; um método fácil de usar, que
recomendo, é lançar mão do pacote requests ( http://docs.python-
requests.org ).

Para encontrar os últimos 30 problemas do pandas no GitHub,
podemos fazer uma requisição HTTP GET usando a biblioteca add-
on requests:

In [62]:
import requests

In [63]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

O método json do objeto Response devolverá um dicionário
contendo o parse dos dados JSON em objetos Python nativos:

In [64]:
data = resp.json()
data[0]['title']

'CLN/TST: Some cleanups in apply.test_invalid_arg'

In [65]:
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])
issues

,number,title,labels,state
0,40688,CLN/TST: Some cleanups in apply.test_invalid_arg,"[{'id': 697792067, 'node_id': 'MDU6TGFiZWw2OTc...",open
1,40687,ENH: Add nullable dtypes to read_csv,[],open
2,40686,ENH: Upgrade Pandas SQLAlchemy code to be comp...,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open
3,40685,BUG: idxmin and idxmax fail for groupby of dec...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
4,40684,BUG: round_trip_pickle for lib.no_default,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
5,40683,Agg functions for df not respecting numeric on...,"[{'id': 1049312478, 'node_id': 'MDU6TGFiZWwxMD...",open
6,40682,BUG: DataFrame from sqlalchemy 1.4 `Query` mis...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
7,40681,REF: simplify dispatch in groupby.ops,"[{'id': 233160, 'node_id': 'MDU6TGFiZWwyMzMxNj...",open
8,40680,DOC: Additional items for the cheat sheet,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
9,40679,TST: [ArrowStringArray] more parameterised tes...,"[{'id': 57522093, 'node_id': 'MDU6TGFiZWw1NzUy...",open


### Interagindo com bancos de dados
Em um ambiente de negócios, a maior parte dos dados talvez não
esteja armazenada em arquivos-texto nem em arquivos Excel.
Bancos de dados relacionais baseados em SQL (como SQL Server,
PostgreSQL e MySQL) são amplamente usados, e muitos bancos
de dados alternativos têm se tornado bem populares. A escolha do
banco de dados em geral depende das necessidades de
desempenho, de integridade dos dados e de escalabilidade de uma
aplicação.

Carregar dados de SQL para um DataFrame é razoavelmente
simples, e o pandas tem algumas funções para simplificar o
processo. Como exemplo, criarei um banco de dados SQLite usando
o driver embutido sqlite3 de Python:

In [67]:
import sqlite3

In [68]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
c REAL, d INTEGER
);
"""

con = sqlite3.connect('mydataBase.sqlite')
con.execute(query)

In [69]:
con.commit()

Em seguida, inserimos algumas linhas de dados:

In [70]:
data = [('Atlanta', 'Georgia', 1.25, 6), 
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

stmt = "INSERT INTO test VALUES(?,?,?,?)"

In [71]:
con.executemany(stmt, data)

In [72]:
con.commit()

A maioria dos drivers de SQL de Python (PyODBC, psycopg2,
MySQLdb, pymssql etc.) devolve uma lista de tuplas ao selecionar
dados de uma tabela:

In [73]:
cursor = con.execute('select * from test')

In [74]:
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

Podemos passar a lista de tuplas para o construtor de DataFrame,
mas também precisaremos dos nomes das colunas, contidos no
atributo *description* do cursor:

In [75]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [76]:
pd.DataFrame(rows, columns=[i[0] for i in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
